In [120]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from adjustText import adjust_text
import scipy.stats as stats

# 1. Se cargan el conjunto de datos

In [121]:
# Define la ruta base del proyecto (la raíz del proyecto)
# Esto asume que el notebook está en la carpeta notebooks

BASE_PATH = os.path.abspath('..')  # Esto asume que el notebook está en la carpeta notebooks

RAW_DATA_PATH = os.path.join(BASE_PATH, 'data', 'raw')
PROCESSED_DATA_PATH = os.path.join(BASE_PATH, 'data', 'processed')

FILE_NAME = 'bank-additional-full.csv'
X_TRAIN = 'X_train.csv'
X_TEST = 'X_test.csv'
X_VALID = 'X_valid.csv'

Y_TRAIN = 'y_train.csv'
Y_TEST = 'y_test.csv'
Y_VALID = 'y_valid.csv'

# Carga un archivo CSV de la carpeta raw (como ejemplo)
CSV_FILE = os.path.join(RAW_DATA_PATH, FILE_NAME)

data = pd.read_csv(CSV_FILE, sep=';')

# Muestra las primeras filas para confirmar que los datos se cargaron correctamente
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


# 2. Normalizar el nombre de las columnas y clases de categorias

In [122]:
data.columns = data.columns.str.lower().str.replace(r'\W+', '_', regex=True)

In [123]:
data_categorical = data.select_dtypes(include=['category', 'object'])
categorical_columns = list(data_categorical.columns)

data_numerical = data.select_dtypes(include=['int64', 'float64'])
numerical_columns = list(data_numerical.columns)

In [124]:
for col in categorical_columns:
    # Convertir a minúsculas y reemplazar signos de puntuación con '_'
    data[col] = data[col].str.lower().str.replace(r'\W+', '_', regex=True)

In [125]:
data.loc[:, 'job'] = data['job'].str.rstrip('_')

data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,56,housemaid,married,basic_4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high_school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high_school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin,married,basic_6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high_school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional_course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue_collar,married,professional_course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university_degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional_course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


# 3. Inputación de valores nulos o ausentes

# 3.1. Variables categóricas

In [126]:
def cross_tab(df, col1, col2):
    edu = list(df[col2].unique())
    dataframes = []
    for item_edu in edu:
        datae = df[df[col2] == item_edu]
        dataejob = datae.groupby(col1).count()[col2]
        dataframes.append(dataejob)
    result = pd.concat(dataframes, axis=1)
    result.columns = edu
    result = result.fillna(0)
    return result

In [127]:
cross_tab(data, 'job', 'education')

,basic_4y,high_school,basic_6y,basic_9y,professional_course,unknown,university_degree,illiterate
job,,,,,,,,
admin,77,3329,151,499,363,249,5753,1.0
blue_collar,2318,878,1426,3623,453,454,94,8.0
entrepreneur,137,234,71,210,135,57,610,2.0
housemaid,474,174,77,94,59,42,139,1.0
management,100,298,85,166,89,123,2063,0.0
retired,597,276,75,145,241,98,285,3.0
self_employed,93,118,25,220,168,29,765,3.0
services,132,2682,226,388,218,150,173,0.0
student,26,357,13,99,43,167,170,0.0


In [128]:
data['job'][data['age'] > 60].value_counts()

job
retired          678
housemaid         54
admin             47
technician        34
management        30
unknown           21
blue_collar       20
self_employed      9
entrepreneur       8
unemployed         7
services           2
Name: count, dtype: int64

In [129]:
data.loc[(data['age'] > 60) & (data['job']=='unknown'), 'job'] = 'retired'
data.loc[(data['education']=='unknown') & (data['job']=='management'), 'education'] = 'university_degree'
data.loc[(data['education']=='unknown') & (data['job']=='services'), 'education'] = 'high_school'
data.loc[(data['education']=='unknown') & (data['job']=='housemaid'), 'education'] = 'basic_4y'
data.loc[(data['job'] == 'unknown') & (data['education']=='basic_4y'), 'job'] = 'blue_collar'
data.loc[(data['job'] == 'unknown') & (data['education']=='basic_6y'), 'job'] = 'blue_collar'
data.loc[(data['job'] == 'unknown') & (data['education']=='basic_9y'), 'job'] = 'blue_collar'
data.loc[(data['job']=='unknown') & (data['education']=='professional_course'), 'job'] = 'technician'

In [130]:
job_housing = cross_tab(data, 'job', 'housing')
job_loan = cross_tab(data, 'job', 'loan')

In [131]:
def fill_housing(df, job_housing):
    """Function for imputation via cross-tabulation to fill missing values for the 'housing' categorical feature"""
    jobs = ['housemaid', 'services', 'admin', 'blue_collar', 'technician', 'retired',
            'management', 'unemployed', 'self_employed', 'entrepreneur', 'student']
    for j in jobs:
        ind = df[np.logical_and(np.array(data['housing']=='unknown'),np.array(data['job']==j))].index
        mask = np.random.rand(len(ind))<((job_housing.loc[j]['no'])/(job_housing.loc[j]['no']+job_housing.loc[j]['yes']))
        ind1 = ind[mask]
        ind2 = ind[~mask]
        df.loc[ind1, "housing"] = 'no'
        df.loc[ind2, "housing"] = 'yes'
    return df

In [132]:
def fill_loan(df, job_loan):
    """Function for imputation via cross-tabulation to fill missing values for the 'loan' categorical feature"""
    jobs = ['housemaid', 'services', 'admin', 'blue_collar', 'technician', 'retired',
            'management', 'unemployed', 'self_employed', 'entrepreneur', 'student']
    for j in jobs:
        ind = data[np.logical_and(np.array(df['loan']=='unknown'),np.array(df['job']==j))].index
        mask = np.random.rand(len(ind))<((job_loan.loc[j]['no'])/(job_loan.loc[j]['no']+job_loan.loc[j]['yes']))
        ind1 = ind[mask]
        ind2 = ind[~mask]
        df.loc[ind1,"loan"] = 'no'
        df.loc[ind2,"loan"] = 'yes'
    return df

In [133]:
data = fill_housing(data, job_housing)
data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,56,housemaid,married,basic_4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high_school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high_school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin,married,basic_6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high_school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional_course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue_collar,married,professional_course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university_degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional_course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [134]:
data = fill_loan(data, job_loan)
data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,56,housemaid,married,basic_4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high_school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high_school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin,married,basic_6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high_school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional_course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue_collar,married,professional_course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university_degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional_course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


# 3. Tratamiento de Outliers

In [135]:
# Lista para almacenar resultados
outliers_list = []

# Calcular outliers para cada columna numérica
for col in numerical_columns:
    # Calcular Q1 y Q3
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    
    # Calcular IQR
    IQR = Q3 - Q1
    
    # Definir los límites de outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Filtrar valores atípicos
    outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)][col]
    
    # Calcular el número y el porcentaje de outliers
    num_outliers = outliers.count()
    outliers_percentage = (num_outliers / data[col].count()) * 100
    
    # Agregar los resultados a la lista
    outliers_list.append({
        'Variable': col,
        'Num Outliers': num_outliers,
        'Outliers Percentage': outliers_percentage
    })

# Convertir la lista a dfFrame
outliers_summary = pd.DataFrame(outliers_list).round(2)

# Mostrar el resumen de outliers
print(outliers_summary)

         Variable  Num Outliers  Outliers Percentage
0             age           469                 1.14
1        duration          2963                 7.19
2        campaign          2406                 5.84
3           pdays          1515                 3.68
4        previous          5625                13.66
5    emp_var_rate             0                 0.00
6  cons_price_idx             0                 0.00
7   cons_conf_idx           447                 1.09
8       euribor3m             0                 0.00
9     nr_employed             0                 0.00


In [136]:
pd.crosstab(data['pdays'], data['poutcome'], values=data['age'], aggfunc='count', normalize=True)

poutcome,failure,nonexistent,success
pdays,,,
0,0.000000,0.000000,0.000364
1,0.000000,0.000000,0.000631
2,0.000000,0.000000,0.001481
3,0.000097,0.000000,0.010561
4,0.000049,0.000000,0.002816
5,0.000097,0.000000,0.001020
6,0.000607,0.000000,0.009396
7,0.000364,0.000000,0.001093
8,0.000146,0.000000,0.000291


In [137]:
data.loc[:, 'pdays_missing'] = 0
data.loc[:, 'pdays_less_5'] = 0
data.loc[:, 'pdays_greater_15'] = 0
data.loc[:, 'pdays_bet_5_15'] = 0

data.loc[:, 'pdays_missing'] = np.where(data['pdays'] == 999, 1, 0)
data.loc[:, 'pdays_less_5'] = np.where(data['pdays'] < 5, 1, 0)
data.loc[:, 'pdays_greater_15'] = np.where((data['pdays'] > 15) & (data['pdays'] < 999), 1, 0)
data.loc[:, 'pdays_bet_5_15'] = np.where((data['pdays'] >= 5) & (data['pdays'] <= 15), 1, 0)

In [138]:
data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,pdays_missing,pdays_less_5,pdays_greater_15,pdays_bet_5_15
0,56,housemaid,married,basic_4y,no,no,no,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,1,0,0,0
1,57,services,married,high_school,unknown,no,no,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,1,0,0,0
2,37,services,married,high_school,no,yes,no,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,1,0,0,0
3,40,admin,married,basic_6y,no,no,no,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,1,0,0,0
4,56,services,married,high_school,no,no,yes,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional_course,no,yes,no,cellular,nov,fri,...,-1.1,94.767,-50.8,1.028,4963.6,yes,1,0,0,0
41184,46,blue_collar,married,professional_course,no,no,no,cellular,nov,fri,...,-1.1,94.767,-50.8,1.028,4963.6,no,1,0,0,0
41185,56,retired,married,university_degree,no,yes,no,cellular,nov,fri,...,-1.1,94.767,-50.8,1.028,4963.6,no,1,0,0,0
41186,44,technician,married,professional_course,no,no,no,cellular,nov,fri,...,-1.1,94.767,-50.8,1.028,4963.6,yes,1,0,0,0


# 4. Generar conjuntos de train, valid y test

In [139]:
data.drop('pdays', axis=1, inplace=True)

In [140]:
data_train = data.drop('y', axis=1)
data_target = data[['y']]

In [141]:
original_count = len(data_train)
training_size = 0.60
test_size = (1 - training_size) / 2


training_count = int(original_count * training_size)
test_count = int(original_count * test_size)
validation_count = original_count - training_count - test_count

print('train:',training_count,'- test:',test_count,'- validation_count:',original_count)

train: 24712 - test: 8237 - validation_count: 41188


In [142]:
from sklearn.model_selection import train_test_split

X_train, rest_x, y_train, rest_y = train_test_split(data_train, data_target, train_size=training_count, stratify=data_target)

X_test, X_valid, y_test, y_valid = train_test_split(rest_x, rest_y, train_size=test_count, stratify=rest_y)

print(len(X_train), len(X_test), len(X_valid))

24712 8237 8239


In [143]:
X_train.shape, X_test.shape, X_valid.shape

((24712, 23), (8237, 23), (8239, 23))

# 5. Crear pipeline

In [152]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import RobustScaler
from category_encoders import BinaryEncoder
from sklearn.preprocessing import LabelEncoder

import sklearn
sklearn.set_config(transform_output="pandas")

In [145]:
encoding  = ColumnTransformer(
    transformers=[
        ("binary_encode", 
         BinaryEncoder(), 
         [
            'job', 'marital', 'education', 'default', 'housing', 
            'loan', 'contact', 'month', 'day_of_week', 'poutcome'             
          ]),  
    ]
)

In [146]:
scaler = ColumnTransformer([
    (
        "scaler", 
        RobustScaler(), 
        [
            'age', 'duration', 'campaign', 'previous', 'emp_var_rate',
            'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed',      
        ]
    )
])

In [147]:
passthrough = ColumnTransformer(
    transformers=[
        ("passthrough", "passthrough", 
         [
            'pdays_missing', 'pdays_less_5', 
            'pdays_greater_15', 'pdays_bet_5_15'
        ])
    ],
)

In [148]:
feature_engineering_pipeline = pipe = Pipeline(
    [
        (
            "features",
            FeatureUnion(
                [
                    ("categorical", encoding),
                    ("scaled", scaler),
                    ("pass", passthrough),
                ]
            ),
        )
    ]
)


In [158]:
label_encoder = LabelEncoder()

y_train_transformed = pd.DataFrame(label_encoder.fit_transform(y_train), columns=['y'])
y_test_transformed = pd.DataFrame(label_encoder.fit_transform(y_test), columns=['y'])
y_valid_transformed = pd.DataFrame(label_encoder.fit_transform(y_valid), columns=['y'])

e:\algorithmic_trading\miniconda3\envs\NoCountry\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
e:\algorithmic_trading\miniconda3\envs\NoCountry\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
e:\algorithmic_trading\miniconda3\envs\NoCountry\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [149]:
X_train_transformed = feature_engineering_pipeline.fit_transform(X_train)
X_test_transformed = feature_engineering_pipeline.fit_transform(X_test)
X_valid_transformed = feature_engineering_pipeline.fit_transform(X_valid)

# 6. Exportar conjunto de datos

In [159]:
X_train_transformed.to_csv(os.path.join(PROCESSED_DATA_PATH, X_TRAIN), index=None)
X_test_transformed.to_csv(os.path.join(PROCESSED_DATA_PATH, X_TEST), index=None)
X_valid_transformed.to_csv(os.path.join(PROCESSED_DATA_PATH, X_VALID), index=None)

y_train_transformed.to_csv(os.path.join(PROCESSED_DATA_PATH, Y_TRAIN), index=None)
y_test_transformed.to_csv(os.path.join(PROCESSED_DATA_PATH, Y_TEST), index=None)
y_valid_transformed.to_csv(os.path.join(PROCESSED_DATA_PATH, Y_VALID), index=None)